In [2]:
path = "C:\\Users\\Michelle\\Documents\\Cummins\\Sem6\\JOCPython\\DataCompression_docs\\"

**Lossy compression is implemented here**<br>

To check whether an image is 8-bit, right click on image file, go to properties, then click details, you will see dimension and bit depth. Bit depth should be 8 bit.<br>

When we open the image using Image.open(), the following will happen. PIL.Image.open(fp, mode='r') opens and identifies the given image file. This is a lazy operation; this function identifies the file, but the file remains open and the actual image data is not read from the file until you try to process the data (or call the load() method). Therefore, im.load() is
required.<br>
np.asanyarray(Image.open('Lena1-8bitImage.png')) is used to convert/see the image as an array<br>
img = Image.new(im.mode, im.size) is to create a new image of same size and mode as object im.<br>

Image compression:
- First decide on compression value, here it is decided that 8 bit to 3 bit compression is to be done.
- The original image is taking 8-bit per pixel (2^8=256) : 0-255
    - for compression: we will only use 3 bit(2^3=8). ie: 8 bit to 3 bit mapping
- To compress 8-bit information to 3-bit, divide 2^8 by 2^3. This gives 2^5 = 32 i.e.: 0-31 range of values.
- therefore, if a pixel value from original image lies between 0-31, then assign 0. This means, we group 32 pixels of value range 0-31 with the same value. 
   - if a pixel value from original image lies between 32-63, then assign 1
   - if value is between 64-95, assign 2
   - if value is between 96-127, assign 3
   - if value is between 128-159, assign 4
   - if between 160-191, assign 5
   - if between 192-223, assign 6
   - if between 224-255, assign 7
- If you have observed, we are actually expressing 8 bit image in terms of 3 bit image (2^3, 0-7 range).
- But by grouping 32 values and assigning them same single value, we are losing the details of those pixels, therefore, the compression is lossy type.
- *for i in range(img.size[0]):*<br>
    &emsp; *for j in range(img.size[1]):*<br>
  Here, size[0]: width of the image (or Row in terms of matrix) and size[1]: height of the image (or Column in terms of matrix) When checked for Lena1-8bitImage.png using print(im.size), it gives (256,256). So there are 256 rows and 256 columns. Here, you divide, 256 into group of 32 for compression.
- Then save the compressed image and display as array using numpy.
- Check the size of the original image (48KB) and compressed image (16KB) when Lena1-8bitImage.png used with 8 bit to 3 bit compression.


In [11]:
import numpy as np
from PIL import Image

img_old = Image.open(path+"Lena1-8bitImage.png")
pixel_old = img_old.load()

img_new = Image.new(img_old.mode, img_old.size) 
pixel_new = img_new.load()

'''
original image is taking 8-bit per pixel (2^8=256)
for compression: we will only use 3 bit(2^3=8). ie: 8 bit to 3 bit mapping
# how to do? divide 2^8 by 2^3=2^5
# therefore, for any no between 0-31 assign 0, 32-63==>1; 64-95==>2; 96-127==>3 
128-159==>4; 160-191==>5; 192-223==>6;224-255==>7
'''

for i in range(img_old.size[0]):
    for j in range(img_old.size[1]):
        if pixel_old[i,j] > 0 and pixel_old[i,j] <= 31: pixel_new[i,j] = 0
        elif pixel_old[i,j] > 31 and pixel_old[i,j] <= 63: pixel_new[i,j] = 1
        elif pixel_old[i,j] > 63 and pixel_old[i,j] <= 95: pixel_new[i,j] = 2
        elif pixel_old[i,j] > 95 and pixel_old[i,j] <= 127: pixel_new[i,j] = 3
        elif pixel_old[i,j] > 127 and pixel_old[i,j] <= 159: pixel_new[i,j] = 4
        elif pixel_old[i,j] > 159 and pixel_old[i,j] <= 191: pixel_new[i,j] = 5
        elif pixel_old[i,j] > 191 and pixel_old[i,j] <= 223: pixel_new[i,j] = 6
        elif pixel_old[i,j] > 223 and pixel_old[i,j] <= 255: pixel_new[i,j] = 7

img_new.save(path+'Lena1-8bitImage-output.png')

In [13]:
ImgasArray_old = np.asanyarray(Image.open(path+'Lena1-8bitImage.png'))
print(ImgasArray_old)  

[[22 22  7 ... 49 49  2]
 [22 22  7 ... 52 49  2]
 [29  0  1 ... 13 15 69]
 ...
 [30 18 26 ... 54 46 53]
 [31 18 18 ... 32 28 19]
 [25 17 18 ... 28 28 40]]


In [14]:
ImgasArray_new = np.asanyarray(Image.open(path+'Lena1-8bitImage-output.png'))  #to see as matrix, try this
print(ImgasArray_new)  

[[0 0 0 ... 1 1 0]
 [0 0 0 ... 1 1 0]
 [0 0 0 ... 0 0 2]
 ...
 [0 0 0 ... 1 1 1]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 1]]


In [64]:
#compress from old bits to new bits  
def compress(file, old, new): 
    img_old = Image.open(file)
    pixel_old = img_old.load()
    img_new = Image.new(img_old.mode, img_old.size) 
    pixel_new = img_new.load()
    step = pow(2,old)/pow(2,new)
    for i in range(img_old.size[0]):
        for j in range(img_old.size[1]):
            k = 0
            while(True):
                if pixel_old[i,j] > (step*k)-1 and pixel_old[i,j] <= (step*(k+1))-1: 
                    pixel_new[i,j] = k
                    break
                elif step*(k+1) == 256: break
                k+=1
    file = file.replace('.png',"")
    img_new.save(f'{file}-compressed_{old}To{new}.png')
    ImgasArray = np.asanyarray(Image.open(f'{file}-compressed_{old}To{new}.png'))
    print(ImgasArray)

In [65]:
compress((path+'Lena1-8bitImage.png'), 8, 3)

[[0 0 0 ... 1 1 0]
 [0 0 0 ... 1 1 0]
 [0 0 0 ... 0 0 2]
 ...
 [0 0 0 ... 1 1 1]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 1]]


In [66]:
compress((path+'Lena1-8bitImage.png'), 8, 4)

[[1 1 0 ... 3 3 0]
 [1 1 0 ... 3 3 0]
 [1 0 0 ... 0 0 4]
 ...
 [1 1 1 ... 3 2 3]
 [1 1 1 ... 2 1 1]
 [1 1 1 ... 1 1 2]]


In [68]:
compress((path+'Lena1-8bitImage.png'), 8, 6)

[[ 5  5  1 ... 12 12  0]
 [ 5  5  1 ... 13 12  0]
 [ 7  0  0 ...  3  3 17]
 ...
 [ 7  4  6 ... 13 11 13]
 [ 7  4  4 ...  8  7  4]
 [ 6  4  4 ...  7  7 10]]


In [70]:
compress((path+'img1.jpg'), 8, 6)

[[38 38 38 ... 38 38 38]
 [38 38 38 ... 38 38 38]
 [38 38 38 ... 38 38 38]
 ...
 [51 51 51 ... 48 48 48]
 [51 51 51 ... 47 48 48]
 [51 51 51 ... 48 48 48]]
